# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [4]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [5]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [43]:
total_df = data.groupby(['CustomerID', 'ProductName']).sum().reset_index()
total_df

,CustomerID,ProductName,SalesID,ProductID,Quantity
0,33,Apricots - Dried,172592,324,1
1,33,Assorted Desserts,5363705,219,1
2,33,Bandage - Flexible Neon,6299165,273,1
3,33,"Bar Mix - Pina Colada, 355 Ml",3147536,281,1
4,33,"Beans - Kidney, Canned",6507033,300,1
...,...,...,...,...,...
63623,98200,Vol Au Vents,5160346,272,50
63624,98200,Wasabi Powder,4438427,80,25
63625,98200,Wine - Fume Blanc Fetzer,6226060,76,25
63626,98200,Wine - Hardys Bankside Shiraz,266676,360,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [36]:
products = total_df.pivot(index='ProductName', columns='CustomerID', values='Quantity')
products = products.fillna(0)
products

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [119]:
similarity_matrix = squareform(pdist(products.T, 'euclidean'))
similarity_matrix

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [120]:
dist_matrix = pd.DataFrame(1/1+similarity_matrix, index=products.columns, columns=products.columns)
dist_matrix.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,12.916375,11.488088,12.224972,12.401754,12.090537,13.409674,12.045361,12.269428,12.489125,...,207.871941,214.180675,226.656819,199.232187,231.913404,221.501701,218.188858,229.628520,240.000000,230.773802
200,12.916375,1.000000,12.747340,13.083046,13.569805,13.288206,13.165525,13.083046,12.874342,13.000000,...,207.310446,213.635839,225.697575,198.139544,231.952376,221.202180,216.728997,229.010965,240.037654,230.704158
264,11.488088,12.747340,1.000000,12.489125,12.224972,12.445523,13.000000,12.401754,12.180340,12.747340,...,207.387984,213.946003,226.435135,198.600607,231.371439,220.136943,217.612557,229.081126,239.266657,230.773802
356,12.224972,13.083046,12.489125,1.000000,13.083046,12.789826,13.328828,12.135529,12.958261,13.165525,...,207.649462,214.082144,226.452878,198.494304,232.038958,220.952268,218.437347,229.098663,239.493186,230.464594
412,12.401754,13.569805,12.224972,13.083046,1.000000,12.704700,13.328828,12.135529,12.789826,12.747340,...,207.900942,212.679002,226.572605,198.630969,231.614397,220.733930,218.446545,228.997807,239.396728,229.927936


In [107]:
customer_200= dist_matrix[200].sort_values(ascending=False).head()
customer_200

CustomerID
92255    249.050398
97324    247.452835
95017    244.903670
92492    241.185345
91777    240.284350
Name: 200, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [108]:
select = total_df[total_df.CustomerID.isin(customer_200.index)]
select

,CustomerID,ProductName,SalesID,ProductID,Quantity
59478,91777,"Appetizer - Mini Egg Roll, Shrimp",4813508,327,24
59479,91777,Banana - Leaves,6398872,271,24
59480,91777,Banana Turning,2449383,304,24
59481,91777,Bandage - Fexible 1x3,3948697,93,24
59482,91777,Bandage - Flexible Neon,4839887,273,24
...,...,...,...,...,...
62914,97324,Wine - Redchard Merritt,397668,425,25
62915,97324,Wine - Vineland Estate Semi - Dry,3444727,427,25
62916,97324,"Wine - White, Colubia Cresh",5121436,44,25
62917,97324,Wonton Wrappers,5905046,311,25


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [78]:
product_select = select.groupby('ProductName').sum().sort_values(by='Quantity', ascending=False)
product_select.head()

,CustomerID,SalesID,ProductID,Quantity
ProductName,,,,
"Beef - Tenderlion, Center Cut",279286,12597850,1760,122
"Turnip - White, Organic",279286,14815578,260,122
Squid U5 - Thailand,373848,15224619,130,121
Muffin Chocolate Individual Wrap,187272,9632176,1724,98
Pastry - Butterscotch Baked,187509,11832037,236,98


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [93]:
temp = product_select.merge(products, on='ProductName')
temp.head(3)

,CustomerID,SalesID,ProductID,Quantity,33,200,264,356,412,464,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
"Beef - Tenderlion, Center Cut",279286,12597850,1760,122,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0
"Turnip - White, Organic",279286,14815578,260,122,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Squid U5 - Thailand,373848,15224619,130,121,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
temp['Quantity'][temp[200]==0].sort_values(ascending=False).head()

ProductName
Beef - Tenderlion, Center Cut       122
Muffin Chocolate Individual Wrap     98
Pastry - Butterscotch Baked          98
Clam Nectar                          98
Onion Powder                         98
Name: Quantity, dtype: int64

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [123]:
dict ={}
customers =[c for c in set(total_df.CustomerID)]

for c in customers:
    distances = dist_matrix[c].sort_values(ascending=False).head()
    similar = total_df[total_df.CustomerID.isin(distances.index)]
    grouped = similar.groupby('ProductName').sum().sort_values(by='Quantity', ascending=False)
    merged = grouped.merge(products, on='ProductName')
    dict[c]= [i for i in merged[merged[c]==0].sort_values(by='Quantity', ascending=False).head().index]


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [125]:
product_recommend = pd.DataFrame(dict).T
product_recommend.head()

,0,1,2,3,4
83973,"Tart Shells - Sweet, 4",Beer - Original Organic Lager,Crab - Imitation Flakes,Tahini Paste,Rum - Mount Gay Eclipes
49159,Soup Campbells - Italian Wedding,Pastry - Butterscotch Baked,Clam Nectar,"Turnip - White, Organic",Table Cloth 81x81 White
22536,"Turnip - White, Organic",Pastry - Butterscotch Baked,Clam Nectar,Muffin Chocolate Individual Wrap,Squid U5 - Thailand
18441,"Turnip - White, Organic",Clam Nectar,Pastry - Butterscotch Baked,Squid U5 - Thailand,Table Cloth 81x81 White
59399,Bagel - Plain,Bread - Hot Dog Buns,Beef - Inside Round,Whmis - Spray Bottle Trigger,Muffin Chocolate Individual Wrap


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [128]:
similarity_matrix = squareform(pdist(products.T, 'cosine'))
dist_matrix = pd.DataFrame(1/1+similarity_matrix, index=products.columns, columns=products.columns)

dict_cosine ={}
customers =[c for c in set(total_df.CustomerID)]

for c in customers:
    distances = dist_matrix[c].sort_values(ascending=False).head()
    similar = total_df[total_df.CustomerID.isin(distances.index)]
    grouped = similar.groupby('ProductName').sum().sort_values(by='Quantity', ascending=False)
    merged = grouped.merge(products, on='ProductName')
    dict[c]= [i for i in merged[merged[c]==0].sort_values(by='Quantity', ascending=False).head().index]

In [129]:
product_recommend_cosine = pd.DataFrame(dict).T
product_recommend_cosine.head()

,0,1,2,3,4
83973,Cumin - Whole,Ice Cream Bar - Hageen Daz To,Olives - Stuffed,"Pepper - Black, Whole","Tart Shells - Sweet, 4"
49159,Cornflakes,Tomatoes Tear Drop,Fondant - Icing,Bread - French Baquette,"Orange - Canned, Mandarin"
22536,Fondant - Icing,Crackers Cheez It,Oil - Shortening - All - Purpose,Vinegar - Sherry,Longos - Grilled Salmon With Bbq
18441,Bread - French Baquette,Bandage - Flexible Neon,"Wine - White, Mosel Gold",Table Cloth 62x114 White,Yogurt - French Vanilla
59399,Juice - Apple Cider,"Turnip - White, Organic",Bread - Italian Corn Meal Poly,"Beef - Tenderlion, Center Cut",French Pastry - Mini Chocolate
